In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep
from selenium import webdriver
import codecs
import json
import urllib
import copy
import urllib.request
import urllib.parse
import os
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from multiprocessing.dummy import Pool 
from selenium.webdriver.chrome.options import Options
options =  Options()
options.binary_location="C:\Program Files (x86)\Google\Chrome\Application\chrome.exe"
# chrome driver: https://chromedriver.chromium.org/downloads
driver=webdriver.Chrome(options=options, executable_path="D:\Program Files\chromedriver.exe")

#### all the category

In [ ]:
catgory=['accounting', 'Analytics', 'aws', 'App-Builder', 'Bookmarks', 'Calendar', 'Call-Tracking', 'Contacts', 
         'Customer-Appreciation', 'CRM', 'Customer-Support', 'Dashboards', 'Databases', 'Developer-Tools', 
         'Devices', 'Documents', 'Drip-Emails', 'Ecommerce', 'Education', 'Email', 'Email-Newsletters', 
         'Event-Management', 'Facebook', 'Fax', 'Files', 'Entertainment', 'Forms', 'Fundraising', 'Gaming', 
         'Google', 'HR', 'business-tools', 'marketing', 'Microsoft', 'News', 'Notes', 'Notifications', 
         'Password-Management', 'Payment-Processing', 'Phone', 'Printing',  'Product-Management', 'Project-Management', 
         'Invoices', 'Real-Estate', 'Reports', 'Scheduling', 'Server-Monitoring', 'Signatures', 'Social', 
         'social-marketing', 'Spreadsheets',  'Surveys', 'Team-Chat', 'Team-Collaboration', 'Time-Tracking', 'Todo-Lists', 
         'Transactional-Email', 'Video-Calls', 'Video', 'Webinars', 'cms', 'WordPress', 'zapier-tools', 'Zoho']

#### download all categories html pages
already done

In [ ]:
for cat in catgory:
    url='https://zapier.com/apps/categories/'+cat
    with urllib.request.urlopen(url)as f:
        html=f.read().decode('utf-8')
    with codecs.open("download_zapier/"+cat+".html","w",'utf-8') as fw:
        fw.write(html)
    print(cat)
#     soup=BeautifulSoup(html, "html5lib")

#### download all the service url inside every category
already done

In [ ]:
for cat in catgory:
    print(cat)
    html=codecs.open("download_zapier/"+cat+".html","r",'utf-8').read() 
    soup=BeautifulSoup(html, "html5lib")
    elem = soup.find_all('li',"service-all")
    path=os.path.join("download_zapier",cat)
    if not os.path.isdir(path):
        print("making:",path)
        os.mkdir(path)
    fw=open(os.path.join(path,'urls'),'w')
    print(len(elem))
    for sect in elem:
        name=sect.find_all('a')[0]
        href=name.get('href')
        name=href.split('/')[2]
        url="https://zapier.com"+href
        fw.write(url+"\n")
        print(url)
    fw.flush()
    fw.close()   

#### download every services and extract every applets urls
need to run many times!

In [ ]:
def get_applet_htmls(url):
    driver.get(url)
    btn = driver.find_elements(By.XPATH, '//button[@aria-label="Next Page"]')
    if btn:
        btn=btn[-1]
    else:
        return -1
    url_list=[]
    flag = btn.is_enabled()
    while flag:
        elem = driver.find_elements_by_class_name("zap-template-preview")
        for ele in elem:
            ele.click()
            sleep(0.02)
        html=driver.page_source
        soup=BeautifulSoup(html, "html5lib")
        applet = soup.find_all("div","zap-template-preview--expanded")
        
        if len(applet)==0:
            break
        for apple in applet:
            url=apple.find_all("a","button--secondary")[0].get('href')
            url="https://zapier.com"+url
            url_list.append(url)
#             print(url)
        flag=btn.is_enabled()
        if flag:
            btn.click()
            sleep(3)
        print(len(url_list), end=" ")
    print("length: {}".format(len(url_list)))
    return url_list

#### keep running this, until it finish every service.

In [ ]:
for root, dirs, files in os.walk("download_zapier", topdown=False):
        for name in files:
            if name.endswith('urls'):
                urls=open(os.path.join(root, name),"r")
                for url in urls:
                    try:
#                         url=url.strip()
                        pathname=os.path.join(root,url.split('/')[-2])
#                        
                        if not os.path.isdir(pathname):
                            os.mkdir(pathname)
                        if os.path.isfile(os.path.join(pathname,'url')) and os.stat(os.path.join(pathname,'url')).st_size>10:
                            print(os.stat(os.path.join(pathname,'url')).st_size)
                            print("omit {}".format(pathname), end=" ")
                            continue
                        print("download {}".format(pathname))
                        applet_urls=get_applet_htmls(url)
                        if applet_urls==-1:
                            print("service {} is too new!".format(url))
                            continue
                        
                        fw= open(os.path.join(pathname,'url'),'w')
                        fw.write("\n".join(applet_urls))
                        fw.flush()
                        fw.close()
                        print("write {} to {}".format(len(applet_urls), pathname))
                    except Exception as err:
                        print("{} we need to restart....".format(err))
                with open(os.path.join(root,name),"w")as fw:
                    fw.write("\n".join(urls))
                    fw.flush()
                    fw.close()